# 1. Prep the dataset

In [1]:
# Install and import all the stuff
!pip install datasets
from datasets import load_dataset
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Load Fruits-360 from HuggingFace
dataset = load_dataset("PedroSampaio/fruits-360")

# Split the train data into training set and validation set
train_data = dataset['train']
train_features, val_features, train_labels, val_labels = train_test_split(train_data['image'], train_data['label'], test_size=0.2, random_state=42)

# Convert sets to Tensorflow-compatible type
training_set = Dataset.from_dict({'image': train_features, 'label': train_labels}).to_tf_dataset(
    columns="image",
    label_cols="label",
    batch_size=32,
    shuffle=True,
)
validation_set = Dataset.from_dict({'image': val_features, 'label': val_labels}).to_tf_dataset(
    columns="image",
    label_cols="label",
    batch_size=32,
    shuffle=True,
)

# 2. Initialize WandB (Weights and Bias)
Remember to create your (free) account on [their website](https://wandb.ai/home) first.

Create a project named "produce_recognition".

In [2]:
# Install WandB
!pip install -q wandb
import wandb

# Login to WandB
wandb.login()

# Initialize WandB with your project name and optional settings
wandb.init(project="produce_recognition")    # <<<<<<<<<< UPDATE THIS IF NEEDED <<<<<<<<<<


#!!!!!!!!!!
# NOTE:
# You might need to log in and enter your API key below.
# Follow the prompt generated below!
#!!!!!!!!!!

wandb: Currently logged in as: jeffreyhsu321. Use `wandb login --relogin` to force relogin


# 3. Configure the Model and WandB
Here we define our model.

In [3]:
import numpy as np
import tensorflow as tf
import wandb
from wandb.keras import WandbCallback

# This function will be provided to WandB to run and train our model multiple times using different hyperparameters.
def train():

    # Initialize WandB with your project name and optional settings
    wandb.init(project="produce_recognition")

    # Define and compile your CNN model
    cnn = tf.keras.models.Sequential()

    # Convolutional layer with Batch Normalization
    cnn.add(tf.keras.layers.Conv2D(filters=wandb.config.conv_layers, kernel_size=wandb.config.conv_kernel_size, activation='relu', input_shape=[100, 100, 3]))
    cnn.add(tf.keras.layers.BatchNormalization())

    # MaxPooling layer
    cnn.add(tf.keras.layers.MaxPool2D(pool_size=wandb.config.pool_size, strides=wandb.config.strides))

    # Convolutional layer with Batch Normalization
    cnn.add(tf.keras.layers.Conv2D(filters=wandb.config.conv_layers2, kernel_size=3, activation='relu'))
    cnn.add(tf.keras.layers.BatchNormalization())

    # Flatten layer
    cnn.add(tf.keras.layers.Flatten())

    # Fully connected (Dense) layer with Batch Normalization and Dropout
    cnn.add(tf.keras.layers.Dense(units=wandb.config.dense_layers, activation='relu'))
    cnn.add(tf.keras.layers.BatchNormalization())
    cnn.add(tf.keras.layers.Dropout(0.2))

    cnn.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=wandb.config.learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Output layer
    cnn.add(tf.keras.layers.Dense(units=113, activation='softmax'))

    # Train the model with the WandB callback
    cnn.fit(x=training_set, validation_data=validation_set, epochs=wandb.config.epochs, verbose=1, callbacks=[WandbCallback()])


sweep_id = wandb.sweep(sweep=sweep_config, project="produce_recognition")

Create sweep with ID: 995bi44g
Sweep URL: https://wandb.ai/jeffreyhsu321/produce_recognition/sweeps/995bi44g


Here we define the hyperparameters we want WandB to play with.

**NOTE!:**

**These parameters should be referenced above when defining the model.**


Example: `pool_size=5` should be updated to `pool_size=wandb.config.pool_size`

In [4]:
sweep_config = {
  "method": "random",
  "name": "sweep-113",
  "metric": {"goal": "minimize", "name": "loss"},

  "parameters": {
    "conv_layers": {'min': 16, 'max': 512},
    "conv_layers2": {'min': 16, 'max': 512},

    "conv_kernel_size": {"values": [9]},

    "pool_size": {"values": [4]},
    "strides": {"values": [9]}, # 3, 9

    "dense_layers": {'min': 256, 'max': 1024},

    "epochs": {"values": [4]},
    'learning_rate': {'min': 0.0001, 'max': 0.012},
  }
}

sweep_id = wandb.sweep(sweep=sweep_config, project="produce_recognition")

Create sweep with ID: 7oaa7yt4
Sweep URL: https://wandb.ai/jeffreyhsu321/produce_recognition/sweeps/7oaa7yt4


Run and observe the sweeps!

Click on the links generated above/below to open the sweeps and charts in another tab on wandb.io!

In [5]:
# Run the sweep agent
wandb.agent(sweep_id, function=train, count=50)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: zew84x6k with config:
wandb: 	conv_kernel_size: 9
wandb: 	conv_layers: 162
wandb: 	conv_layers2: 130
wandb: 	dense_layers: 408
wandb: 	epochs: 4
wandb: 	learning_rate: 0.0019832942649855123
wandb: 	pool_size: 4
wandb: 	strides: 9
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/4


Exception in thread ChkStopThrException in thread :
Exception in thread NetStatThr:
IntMsgThrTraceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
:
Traceback (most recent call last):
Traceback (most recent call last):
      File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()self.run()    
self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run

  File "/usr/lib/python3.10/threading.py", line 953, in run
      File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)    self._target(*self._args, **self._kwargs)

self._target(*self._args, **self._kwargs)  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 268, in check_network_status
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 286, in check_stop_status

    

1693/1693 [==============================] - ETA: 0s - loss: 0.3146 - accuracy: 0.9169

...

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
wandb: Adding directory to artifact (/content/wandb/run-20231201_223938-2zqgpmjx/files/model-best)... Done. 0.1s


1693/1693 [==============================] - 49s 29ms/step - loss: 0.1165 - accuracy: 0.9652 - val_loss: 0.0292 - val_accuracy: 0.9905
Epoch 3/4
1693/1693 [==============================] - 47s 28ms/step - loss: 0.0767 - accuracy: 0.9768 - val_loss: 0.0595 - val_accuracy: 0.9859
Epoch 4/4
1693/1693 [==============================] - 47s 28ms/step - loss: 0.0593 - accuracy: 0.9825 - val_loss: 0.0365 - val_accuracy: 0.9878


accuracy,▁▇██
epoch,▁▃▆█
loss,█▂▁▁
val_accuracy,▁███
val_loss,█▁▁▁
accuracy,0.98251
best_epoch,1
best_val_loss,0.02918
epoch,3
loss,0.05929
val_accuracy,0.98781


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x78732d42ad10>> (for post_run_cell):


BrokenPipeError: ignored